# Combine Rename & Mosaic Rasters

[![Open In Colab](https://colab.research.google.com/assets/colab-btn.svg)](https://colab.research.google.com/github/ulfboge/temporal-landcover-vectorizer/blob/main/scripts/python/combined_rename_mosaic.ipynb)

This notebook demonstrates how to rename raster bands and then mosaic the resulting rasters into one output.

We'll:
1. Create directories (using `os.makedirs`).
2. Define functions to rename raster bands.
3. Rename each raster in a specified input directory.
4. Mosaic the renamed rasters into one final file.

In [ ]:
import os
from osgeo import gdal
import glob

# 1) Define directories
data_dir = "/content/drive/MyDrive/earthengine/ndvi"  # Folder containing your original TIFFs
output_folder = os.path.join(data_dir, "ndvi_combined")

# Create directories if they don't exist
os.makedirs(data_dir, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Output folder : {output_folder}")

## 2) Define Utility Functions

### `rename_raster_bands()`
- Opens a raster using GDAL.
- Creates a copy of that raster.
- Renames each band using `SetDescription`.
- Saves the new raster in a specified output path.

### `mosaic_rasters()`
- Uses `gdal.Warp` to merge multiple input rasters into a single output mosaic.

In [ ]:
def rename_raster_bands(input_raster_path, output_raster_path, band_names):
    """
    Renames raster bands for a given input raster, then writes out a new file.

    :param input_raster_path:  Path to the input raster
    :param output_raster_path: Path to the output raster
    :param band_names:         List of strings for new band names
    """
    # Open input raster
    dataset = gdal.Open(input_raster_path, gdal.GA_ReadOnly)
    if not dataset:
        raise RuntimeError(f"Could not open {input_raster_path}")

    # Create output raster (copy)
    driver = gdal.GetDriverByName("GTiff")
    out_dataset = driver.CreateCopy(output_raster_path, dataset, 0)

    # Rename each band using metadata
    for i, name in enumerate(band_names, start=1):
        band = out_dataset.GetRasterBand(i)
        if band:
            band.SetDescription(name)

    # Flush and close
    out_dataset.FlushCache()
    out_dataset = None
    dataset = None

def mosaic_rasters(input_rasters, output_mosaic_path):
    """
    Merges a list of rasters into a single output mosaic using GDAL.

    :param input_rasters:      List of raster file paths
    :param output_mosaic_path: Path to the output mosaic file
    """
    # Use gdal.Warp for mosaic
    gdal.Warp(
        destNameOrDestDS=output_mosaic_path,
        srcDSOrSrcDSTab=input_rasters,
        format="GTiff",
        options=["COMPRESS=LZW"]  # Optional compression
    )
    print(f"Mosaic created: {output_mosaic_path}")

## 3) Rename Raster Bands in All `.tif` Files
In this step:
- We look for all TIFFs in `data_dir`.
- We rename the band(s) to whatever we want (in this example, a single band named `NDVI`).
- Each renamed file is saved in `output_folder` with a `_renamed.tif` suffix.

In [ ]:
input_tifs = glob.glob(os.path.join(data_dir, '*.tif'))
print("Found TIFs:", input_tifs)

# Suppose we only have 1 band that we want to label 'NDVI'
new_band_names = ["NDVI"]

renamed_tifs = []

for in_tif in input_tifs:
    base_name = os.path.basename(in_tif)
    out_tif = os.path.join(output_folder, os.path.splitext(base_name)[0] + "_renamed.tif")

    rename_raster_bands(in_tif, out_tif, new_band_names)
    renamed_tifs.append(out_tif)

print("Renaming complete.")
print("Renamed TIFs:", renamed_tifs)

## 4) Mosaic the Renamed Raster Files
Next, we combine all these renamed TIFs into a single mosaic.

In [ ]:
final_mosaic_path = os.path.join(output_folder, "ndvi_mosaic.tif")

mosaic_rasters(renamed_tifs, final_mosaic_path)
print("All rasters have been renamed and mosaiced into:", final_mosaic_path)

### Done!
Check your Google Drive folder for the `ndvi_combined` subfolder, which will now contain:
1. The renamed individual TIFs (with `_renamed` suffix).
2. The final `ndvi_mosaic.tif`.